In [1]:
import torch, gc, os
import pandas as pd

from models.load import TrainedModels, get_trained_model
from data.dataset import ReflacxDataset, collate_fn
from utils.transforms import get_transform
## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"


In [2]:
gc.collect()
# torch.cuda.memory_summary(device=None, abbreviated=False)


use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print(f"This notebook will running on device: [{device.upper()}]")

if use_gpu:
    torch.cuda.empty_cache()

This notebook will running on device: [CUDA]


In [3]:
labels_cols = [
    "Enlarged cardiac silhouette",
    "Atelectasis",
    "Pleural abnormality",
    "Consolidation",
    "Pulmonary edema",
    #  'Groundglass opacity', # 6th disease.
]

In [4]:
model, train_info = get_trained_model(TrainedModels.custom_with_clinical_drop5, labels_cols, device, include_train_info=True)

{'rpn_nms_thresh': 0.3, 'box_detections_per_img': 6, 'box_nms_thresh': 0.2, 'rpn_score_thresh': 0.0, 'box_score_thresh': 0.05}
c1
None


In [5]:
model

MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu

In [6]:
train_info.best_ap_val_model_path

'val_ar_0_4226_ap_0_2444_test_ar_0_5152_ap_0_2194_epoch18_WithClincal_03-28-2022 17-01-53_custom_with_clinical'

In [7]:
labels_cols = [
    "Enlarged cardiac silhouette",
    "Atelectasis",
    "Pleural abnormality",
    "Consolidation",
    "Pulmonary edema",
    #  'Groundglass opacity', # 6th disease.
]

dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": train_info.model_setup.use_clinical,
    "dataset_mode": "normal",
    "bbox_to_mask": True,
    "labels_cols": labels_cols,
}

test_dataset = ReflacxDataset(
    **dataset_params_dict, split_str="test", transforms=get_transform(train=False),
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn,
)

In [8]:
data = next(iter(test_dataloader))
data = test_dataset.prepare_input_from_data(data, device)

In [10]:
model.eval()
pred = model(*data[:-1])

In [11]:
pred[0]['boxes']

tensor([[ 916.3984, 1493.5072, 2205.9187, 2165.6917],
        [1354.2728,  474.6616, 2177.6750, 2287.1362],
        [ 286.3405, 1684.1877,  507.0537, 2203.4973],
        [ 238.3649,  526.8069, 1036.0875, 2187.8645]], device='cuda:0',
       grad_fn=<StackBackward0>)